## Project 1: Robot Path Planning 

In [71]:
# Import libaries
import math
import altair as alt
import pandas as pd

In [72]:
# Read in input file store start coordinate, end coordinate, and the grid matrix
def read_input_file(file_path):
    with open(file_path, 'r') as file:
        first_line = file.readline().strip()
        start_x, start_y, goal_x, goal_y = map(int, first_line.split())
        grid_matrix = []
        for y in range(29, -1, -1):  
            line = file.readline().strip()
            if line: 
                cell_values = list(map(int, line.split()))
                grid_matrix.append([[x, y, cell_values[x]] for x in range(len(cell_values))])

    return (start_x, start_y), (goal_x, goal_y), grid_matrix

start_coord, goal_coord, grid_matrix = read_input_file("input2.txt")
print(start_coord)
print(goal_coord)

(3, 2)
(11, 13)


In [73]:
flat_grid = [cell for row in grid_matrix for cell in row]
df = pd.DataFrame(flat_grid, columns=['x', 'y', 'cell_value'])

color_scale = alt.Scale(domain=[0, 1, 2, 5], range=['white', 'black', 'red', 'green'])

heatmap = alt.Chart(df).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title='x', values=list(range(0, 51, 1)))),
    y=alt.Y('y:O', axis=alt.Axis(title='y', values=list(range(0, 31, 1))), sort='descending'),
    color=alt.Color('cell_value:N', scale=color_scale)
).properties(
    width=800,
    height=400
)

horizontal_grid = alt.Chart(pd.DataFrame({'y': [i + 0.5 for i in range(31)]})).mark_rule(color='gray', strokeWidth=0.5).encode(
    y=alt.Y('y:O', sort='descending') 
)

vertical_grid = alt.Chart(pd.DataFrame({'x': [i + 0.5 for i in range(51)]})).mark_rule(color='gray', strokeWidth=0.5).encode(
    x='x:O'
)

heatmap = heatmap + horizontal_grid + vertical_grid

heatmap

alt.LayerChart(...)

In [74]:
class Node:
    def __init__(self, coord, parent=None):
        self.coord = coord
        self.parent = parent
        self.g = 0 
        self.h = 0 
        self.f = 0 

actions = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]

def is_legal_move(x, y, grid_dict):
    return grid_dict.get((x, y), 1) == 0 or grid_dict.get((x, y), 1) == 2 or grid_dict.get((x, y), 1) == 5

def euclidean_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def a_star_search(start_coord, goal_coord, grid_dict):
    frontier = []
    visited = set()
    start_node = Node(start_coord)
    start_node.h = euclidean_distance(start_coord, goal_coord)
    start_node.f = start_node.h + start_node.g
    frontier.append(start_node)
    
    while frontier:
        current_node = min(frontier, key=lambda o: o.f)
        frontier.remove(current_node)
        visited.add(current_node.coord)
        
        if current_node.coord == goal_coord:
            path = []
            current = current_node
            while current is not None:
                path.append(current.coord)
                current = current.parent
            return path[::-1]

        for action in actions:
            new_coord = (current_node.coord[0] + action[0], current_node.coord[1] + action[1])
            if not is_legal_move(new_coord[0], new_coord[1], grid_dict) or new_coord in visited:
                continue

            new_node = Node(new_coord, current_node)
            new_node.g = current_node.g + (math.sqrt(2) if action in [(1, 1), (-1, 1), (-1, -1), (1, -1)] else 1)
            new_node.h = euclidean_distance(new_coord, goal_coord)
            new_node.f = new_node.g + new_node.h
            
            if any(node.coord == new_node.coord and node.g <= new_node.g for node in frontier):
                continue
            frontier.append(new_node)
    return None

In [75]:
grid_dict = {(x, y): value for row in grid_matrix for x, y, value in row}
path = a_star_search(start_coord, goal_coord, grid_dict)
if path:
    print(path)
else:
    print("No path found")

[(3, 2), (4, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 10), (7, 10), (8, 11), (9, 11), (10, 12), (11, 13)]


In [76]:
flat_grid = [cell for row in grid_matrix for cell in row]
df = pd.DataFrame(flat_grid, columns=['x', 'y', 'cell_value'])

color_scale = alt.Scale(domain=[0, 1, 2, 5], range=['white', 'black', 'red', 'blue'])

heatmap = alt.Chart(df).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title='x', values=list(range(0, 51, 1)))),
    y=alt.Y('y:O', axis=alt.Axis(title='y', values=list(range(0, 31, 1))), sort='descending'),
    color=alt.Color('cell_value:N', scale=color_scale)
).properties(
    width=800,
    height=400
)

path_df = pd.DataFrame(path, columns=['x', 'y'])
path_map = alt.Chart(path_df).mark_point(color='green', filled=True, size=50).encode(
    x=alt.X('x:O', axis=alt.Axis(title='x', values=list(range(0, 51, 1)))),
    y=alt.Y('y:O', axis=alt.Axis(title='y', values=list(range(0, 31, 1))), sort='descending')
)

horizontal_grid = alt.Chart(pd.DataFrame({'y': [i + 0.5 for i in range(31)]})).mark_rule(color='gray', strokeWidth=0.5).encode(
    y=alt.Y('y:O', sort='descending') 
)

vertical_grid = alt.Chart(pd.DataFrame({'x': [i + 0.5 for i in range(51)]})).mark_rule(color='gray', strokeWidth=0.5).encode(
    x='x:O'
)

heatmap = heatmap + horizontal_grid + vertical_grid + path_map

heatmap

alt.LayerChart(...)